# Importing libraries

In [ ]:
# General purpose
import numpy as np
import pandas as pd

# Libraries for plotting
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt

# Libraries for importing datasets from Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Scikit-Learn libraries
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics

# If Google Colab does not have pygeohash
!pip install pygeohash
import pygeohash as gh

# Authentication for using Google Drive files

In [ ]:
# Configuring Google Drive file loading (run it only once)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1MURORv9iCRNNZtZORoNlv3WtDAYcGYqJ'
id = link.split('=')[1]

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile(id)  
imported_df = pd.read_csv(id, sep=',')

df = imported_df.copy(deep = False)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data preprocessing

In [ ]:
# for every column if it contains just one value (without add Nan to the count) drop it
for col in df:
  if not df[col].value_counts().to_list():
    df.drop(col, axis = 1,inplace = True)

# removig the index
df.drop('Accident_Index',axis = 1, inplace = True)
# removing useless columns (more than 90% of the data have the same value)
df.drop('Special_Conditions_at_Site', axis = 1,inplace = True)
df.drop('Carriageway_Hazards',axis = 1,inplace = True)
df.drop('Pedestrian_Crossing-Human_Control',axis = 1, inplace= True)
# removing duplicate of coordinate system
df.drop('Location_Northing_OSGR',axis = 1, inplace = True)
df.drop('Location_Easting_OSGR',axis = 1,inplace = True)
df.drop('LSOA_of_Accident_Location',axis = 1,inplace = True)
df.drop('1st_Road_Class',axis = 1, inplace = True)
df.drop('1st_Road_Number',axis = 1, inplace = True)
df.drop('2nd_Road_Class',axis = 1, inplace = True)
df.drop('2nd_Road_Number',axis = 1, inplace = True)
df.drop('Police_Force',axis = 1, inplace = True)
df.drop('Local_Authority_(District)',axis = 1,inplace = True)

# removing junctioncontrol because it has 40% of nan values
df.drop('Junction_Control',axis = 1,inplace = True)
# REMOVE ALSO THE FIRST AND SECOND ORDER STUFF BECAUSE THEY ARE USEFUL ONLY FOR JUNCTIONS

# removing the date because we just use the day of the week
df.drop('Date',inplace = True, axis = 1)

# removing 13 nan rows in time, no way to impute it
df.dropna(subset = ['Time'],inplace = True)
df.dropna(subset = ['Did_Police_Officer_Attend_Scene_of_Accident'],inplace = True)

# merging attribute values
df.loc[df['Number_of_Vehicles'] > 2,'Number_of_Vehicles'] = '3+'
df.loc[df['Number_of_Vehicles'] == 2,'Number_of_Vehicles'] = 'two'
df.loc[df['Number_of_Vehicles'] == 1,'Number_of_Vehicles'] = 'one'
df.loc[(df['Road_Type'] != 'Single carriageway') & (df['Road_Type'] != 'Dual carriageway'),'Road_Type'] = 'Other'
df.loc[df['Pedestrian_Crossing-Physical_Facilities'] != 'No physical crossing within 50 meters', 'Pedestrian_Crossing-Physical_Facilities'] = 'Physical Crossing present'
df.loc[(df['Weather_Conditions'] != 'Fine without high winds') & (df['Weather_Conditions'] != 'Raining without high winds'), 'Weather_Conditions'] = 'Other'
df.loc[(df['Road_Surface_Conditions'] != 'Dry') & (df['Road_Surface_Conditions'] != 'Wet/Damp'),'Road_Surface_Conditions'] = 'Extreme_condition' 
df.loc[(df['Number_of_Casualties'] != 1) & (df['Number_of_Casualties'] != 2),'Number_of_Casualties'] = '3+'
df.loc[df['Number_of_Casualties'] == 2,'Number_of_Casualties'] = 'two'
df.loc[df['Number_of_Casualties'] == 1,'Number_of_Casualties'] = 'one' 
df.loc[(df['Light_Conditions'] == 'Daylight: Street light present') | (df['Light_Conditions'] == 'Darkness: Street lights present and lit'),'Light_Conditions'] = 'Proper lightning'
df.loc[(df['Light_Conditions'] == 'Darkness: Street lights present but unlit') | (df['Light_Conditions'] == 'Darkeness: No street lighting'),'Light_Conditions'] = 'Insufficient lightning'
df.loc[df['Light_Conditions'] == 'Darkness: Street lighting unknown','Light_Conditions'] = 'Unknown'
df.loc[df['Speed_limit'] == 10,'Speed_limit'] = 20
hours = pd.to_datetime(df['Time'], format='%H:%M').dt.hour
df['Time'] = pd.cut(hours, 
                    bins=[0,6,12,18,24], 
                    include_lowest=True, 
                    labels=['Midnight','Morning','Evening','Night'])

In [ ]:
# DISCRETIZE COLUMNS
df['Day_of_Week'] = df['Day_of_Week'].apply(str) # categorical
df['Time'] = df['Time'].apply(str) # categorical
df['Road_Type'] = df['Road_Type'].apply(str) # categorical
df['Speed_limit'] = df['Speed_limit'].apply(str) # categorical
df['Urban_or_Rural_Area'] = df['Urban_or_Rural_Area'].apply(str) # categorical
df['Year'] = df['Year'].apply(str) # categorical

In [ ]:
#DISCRETIZE Longitude and Latitude using Geohash
df['geohash']=df.apply(lambda x: gh.encode(x.Latitude, x.Longitude, precision=3), axis=1)
df.drop('Longitude',axis = 1, inplace = True)
df.drop('Latitude',axis = 1, inplace = True)

features = df.drop('Accident_Severity',axis = 1).copy()

Apply one hot encoding

In [ ]:
# Save categorical variables
categorical_features = features.select_dtypes(include = ['object','category']).columns

# Categorical variables One Hot Encoding
categorical_transformer = Pipeline(steps = [
    ('onehot', preprocessing.OneHotEncoder())
])

numerical_features = features.select_dtypes(exclude = ['object','category']).columns


numerical_transformer = Pipeline(steps = [
    ('scaler',preprocessing.StandardScaler())
])

# Aggiungo le pipeline in un unico processo preprocessor
preprocessor = ColumnTransformer( transformers = [
    ('num',numerical_transformer, numerical_features),                                 
    ('cat',categorical_transformer, categorical_features)
])

# Create a model that contains all the possible features --> max variance
transformation = make_pipeline(
    preprocessor
)

transformation.fit(features)

# Dataset downsampling

In [ ]:
# Downsampling

df_fatal = df[df['Accident_Severity'] == 1]
df_serious = df[df['Accident_Severity'] == 2]
df_slight = df[df['Accident_Severity'] == 3]

df_serious_resampled = resample(df_serious, n_samples=len(df_fatal), random_state=0)
df_slight_resamples = resample(df_slight, n_samples=len(df_fatal), random_state=0)

df_downsampled = pd.concat([df_fatal, df_serious_resampled, df_slight_resamples])
df_downsampled = df_downsampled.sample(frac=1).reset_index(drop=True)

In [ ]:
# Downsampling by merging Serious and Fatal

df_fatal = df[df['Accident_Severity'] == 1]
df_serious = df[df['Accident_Severity'] == 2]
df_slight = df[df['Accident_Severity'] == 3]

df_slight_resamples = resample(df_slight, n_samples=len(df_fatal)+len(df_serious), random_state=0)

# merge the fatal and the serious together
df_merged_downsampled = pd.concat([df_fatal, df_serious, df_slight_resamples])
tmp = df_merged_downsampled['Accident_Severity'].copy()
tmp[tmp == 2] = 1
df_merged_downsampled['Accident_Severity'] = tmp
df_merged_downsampled = df_merged_downsampled.sample(frac=1).reset_index(drop=True)


**ATTENTION!**
> DEFINE WHICH DATASET WILL BE USED BY USING ONLY 1 LINE AT A TIME



In [ ]:

X = df_downsampled
#X = df_merged_downsampled

# Main program
## Define the model

In [ ]:
#Use only one clf at a time by commenting and uncommenting the correct lines

#################################### Stohastic SVM ####################################################
#Best parameter for df_downsampled dataset
clf = SGDClassifier(loss="hinge", n_iter_no_change=100, alpha=0.001, max_iter=2000)

#Best parameter for df_merged_downsampled dataset
#clf = SGDClassifier(loss="hinge", n_iter_no_change=100, alpha=0.000001, max_iter = 10000)
########################################################################################################

##################################### RandomForest #####################################################
#Best parameter for df_downsampled dataset
#clf = RandomForestClassifier(n_estimators= 200, max_depth= 30, max_features='auto', criterion='entropy')

#Best parameter for df_merged_downsampled dataset
#clf = RandomForestClassifier(n_estimators= 200, max_depth= 30, max_features='sqrt', criterion='gini')
########################################################################################################


Run the model with K-fold cross validation

In [ ]:
accuracy = precision = recall = fmeasure = count = 0

#number of folds
K = 10

kf = KFold(n_splits=K)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
  count += 1

  X_train = X.iloc[train_index].drop('Accident_Severity',axis = 1).copy()
  X_test = X.iloc[test_index].drop('Accident_Severity',axis = 1).copy()
  Y_train = X.iloc[train_index]['Accident_Severity']
  Y_test = X.iloc[test_index]['Accident_Severity']
  clf.fit(transformation.transform(X_train), Y_train)

  predicted = clf.predict(transformation.transform(X_test))

  accuracy += metrics.accuracy_score(Y_test, predicted)  
  precision += metrics.precision_score(Y_test, predicted, pos_label=1, average = None) 
  recall += metrics.recall_score(Y_test, predicted, pos_label=1, average = None) 
  fmeasure += metrics.f1_score(Y_test, predicted, pos_label=1, average = None)
  print("Fold "+str(count)+" finished")

print("Precision = " + str(precision/K))
print("Accuracy = " + str(accuracy/K))
print("Recall = " + str(recall/K))
print("F-Measure = " + str(fmeasure/K))

## Grid search



Defining train and test datasets

In [ ]:
train_df, test_df = train_test_split(new_df,test_size = 0.2)
X_train = train_df.drop('Accident_Severity',axis = 1).copy()
Y_train = train_df['Accident_Severity'].copy()
X_test = test_df.drop('Accident_Severity',axis = 1).copy()
Y_test = test_df['Accident_Severity'].copy()

Grid search for SGDC

In [ ]:
param_grid = {
    'penalty': ["l2", "l1", "elasticnet"],
    'alpha': [10 ** x for x in range(-7, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
    'n_iter_no_change': [100, 500, 1000]
}
clf = SGDClassifier(loss="hinge",  max_iter=4000)

clf_grid = GridSearchCV(estimator=clf, param_grid=param_grid,
                                    n_jobs=-1, cv=3)

clf_grid.fit(transformation.transform(X_train), Y_train)

clf_grid.best_params_

Grid search for RandomForest

In [ ]:
param_grid = {"n_estimators":[100, 200],
              "max_depth": [30],
              "max_features": ['auto', 'log2', 'sqrt'],
              "criterion": ['gini','entropy']}

clf = RandomForestClassifier()

clf_grid = GridSearchCV(t2, param_grid=param_grid, cv=3, n_jobs = -1)

clf_grid.fit(transformation.transform(X_train),y_train)

clf_grid.best_params_